In [2]:
while True print('Hello world')

SyntaxError: invalid syntax (2884618176.py, line 1)

In [3]:
10 * (1/0)

ZeroDivisionError: division by zero

In [4]:
4 + spam*3

NameError: name 'spam' is not defined

In [5]:
'2' + 2

TypeError: can only concatenate str (not "int") to str

## **Handling Exceptions**

In [6]:
while True:
    try:
        x = int(input("Please enter a number: "))
        break
    except ValueError:
        print("Oops!  That was no valid number.  Try again...")

Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...


In [7]:
while True:
    x = int(input("Please enter a number: "))
    break

ValueError: invalid literal for int() with base 10: 's'

A `try` statement may have more than one `except` clause, to specify handlers for different exceptions. At most one handler will be executed.

An `except` clause may name multiple exceptions as a parenthesized tuple, for example:

```python
... except (RuntimeError, TypeError, NameError):
...     pass
````

***
A class in an `except` clause matches exceptions which are instances of the class itself or one of its derived classes (but not the other way around — an except clause listing a derived class does not match instances of its base classes). For example, the following code will print B, C, D in that order:

In [10]:
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except D:
        print("D")
    except C:
        print("C")
    except B:
        print("B")

B
B
B


***
When an exception occurs, it may have associated values, also known as the exception’s ***arguments***. The presence and types of the arguments depend on the exception type.

The except clause may specify a variable after the exception name. The variable is bound to the exception instance which typically has an `args` attribute that stores the arguments. For convenience, builtin exception types define `__str__()` to print all the arguments without explicitly accessing `.args`.

The exception’s `__str__()` output is printed as the last part (`‘detail’`) of the message for unhandled exceptions.


In [11]:
try:
    raise Exception('spam', 'eggs')
except Exception as inst:
    print(type(inst))    # the exception type
    print(inst.args)     # arguments stored in .args
    print(inst)          # __str__ allows args to be printed directly,
                         # but may be overridden in exception subclasses
    x, y = inst.args     # unpack args
    print('x =', x)
    print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


In [12]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error:", err)
except ValueError:
    print("Could not convert data to an integer.")
except Exception as err:
    print(f"Unexpected {err=}, {type(err)=}")
    raise

OS error: [Errno 2] No such file or directory: 'myfile.txt'


### The most common pattern for handling Exception is to print or log the exception and then re-raise it (allowing a caller to handle the exception as well):

In [13]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error:", err)
except ValueError:
    print("Could not convert data to an integer.")
except Exception as err:
    print(f"Unexpected {err=}, {type(err)=}")
    raise

OS error: [Errno 2] No such file or directory: 'myfile.txt'


---
### The `try … except` statement has an optional `else` clause, which, when present, must follow all ***except clauses***. 
## It is useful for code that must be executed if the try clause does not raise an exception. For example:

In [14]:
for arg in sys.argv[1:]:
    try:
        f = open(arg, 'r')
    except OSError:
        print('cannot open', arg)
    else:
        print(arg, 'has', len(f.readlines()), 'lines')
        f.close()

cannot open --f=/run/user/407832/jupyter/runtime/kernel-v3a2fea45adbb581c772d0e2c5de9722731e562f4a.json


---
The use of the `else` clause is better than adding additional code to the `try clause` because it avoids accidentally catching an exception that wasn’t raised by the code being protected by the `try … except` statement.

Exception handlers do not handle only exceptions that occur immediately in the ***try clause***, but also those that occur inside functions that are called (even indirectly) in the try clause. For example:

In [15]:
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)


Handling run-time error: division by zero


## **Raising Exceptions**

In [16]:
raise NameError('HiThere')

NameError: HiThere

The sole argument to `raise` indicates the exception to be raised. This must be either an exception instance or an exception class (a class that derives from `BaseException`, such as `Exception` or one of its subclasses). 

If an exception class is passed, it will be implicitly instantiated by calling its constructor with no arguments:

In [17]:
raise ValueError  # shorthand for 'raise ValueError()'

ValueError: 

---
If you need to determine whether an exception was raised but don’t intend to handle it, a simpler form of the `raise` statement allows you to re-raise the exception:

In [18]:
try:
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    raise

An exception flew by!


NameError: HiThere

---
### **Exception Chaining**
If an unhandled exception occurs inside an `except` section, it will have the exception being handled attached to it and included in the error message:

In [19]:
try:
    open("database.sqlite")
except OSError:
    raise RuntimeError("unable to handle error")

RuntimeError: unable to handle error

---
### Exception Chaining
### To indicate that an exception is a direct consequence of another, the `raise` statement allows an optional from clause:
```python
# exc must be exception instance or None.
raise RuntimeError from exc
```
This can be useful when you are transforming exceptions. For example:

In [1]:
try:
    open("database.sqlite")
except OSError:
    raise RuntimeError("unable to handle error")

RuntimeError: unable to handle error

In [20]:
def func():
    raise ConnectionError

try:
    func()
except ConnectionError as exc:
    raise RuntimeError('Failed to open database') from exc

RuntimeError: Failed to open database

---
Say you try to convert a string to an int:
```python
try:
    x = int("abc")
except ValueError as exc:
    raise RuntimeError("Could not parse number") from exc
```
---
Output traceback will show:
```python
RuntimeError: Could not parse number
    ...
The above exception was the direct cause of the following exception:
ValueError: invalid literal for int()
```
---
So Python tells you:

> The ValueError caused the RuntimeError.

This “cause” chain makes debugging much easier.

---
### It also allows disabling automatic exception chaining using the `from None` idiom:

In [22]:
try:
    open('database.sqlite')
except OSError:
    raise RuntimeError from None

RuntimeError: 

---
## Defining Clean-up Actions
The `try` statement has another optional clause which is intended to define clean-up actions that must be executed under all circumstances

In [2]:
try:
    raise KeyboardInterrupt
finally:
    print('Goodbye, world!')

Goodbye, world!


KeyboardInterrupt: 

If a `finally` clause is present, the finally clause will execute as the last task before the `try` statement completes. The finally clause runs whether or not the try statement produces an exception. The following points discuss more complex cases when an exception occurs:

* If an exception occurs during execution of the try clause, the exception may be handled by an `except` clause. If the exception is not handled by an except clause, the exception is re-raised after the finally clause has been executed.
* An exception could occur during execution of an except or else clause. Again, the exception is re-raised after the ***finally*** clause has been executed.
* If the finally clause executes a `break`, `continue` or `return` statement, exceptions are not re-raised.
* If the try statement reaches a `break`, `continue` or `return` statement, the finally clause will execute just prior to the `break`, continue or return statement’s execution.
* If a ***finally*** clause includes a return statement, the returned value will be the one from the finally clause’s return statement, not the value from the try clause’s return statement

In [3]:
def bool_return():
    try:
        return True
    finally:
        return False

bool_return()

False

In [6]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")

divide(2, 1)

result is 2.0
executing finally clause


In [7]:
divide(2, 0)

division by zero!
executing finally clause


In [8]:
divide("2", "1")

executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

As you can see, the `finally` clause is executed in any event. The `TypeError` raised by dividing two strings is not handled by the except clause and therefore re-raised after the `finally` clause has been executed.

---
## Raising and Handling Multiple Unrelated Exceptions
There are situations where it is necessary to report several exceptions that have occurred. This is often the case in concurrency frameworks, when several tasks may have failed in parallel, but there are also other use cases where it is desirable to continue execution and collect multiple errors rather than raise the first exception.

The builtin `ExceptionGroup` wraps a list of exception instances so that they can be raised together. It is an exception itself, so it can be caught like any other exception.

In [9]:
def f():
    excs = [OSError('error 1'), SystemError('error 2')]
    raise ExceptionGroup('there were problems', excs)

f()

  + Exception Group Traceback (most recent call last):
  |   File "/nsls2/conda/envs/2025-3.0-py312-tiled/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_976887/1521594820.py", line 5, in <module>
  |     f()
  |   File "/tmp/ipykernel_976887/1521594820.py", line 3, in f
  |     raise ExceptionGroup('there were problems', excs)
  | ExceptionGroup: there were problems (2 sub-exceptions)
  +-+---------------- 1 ----------------
    | OSError: error 1
    +---------------- 2 ----------------
    | SystemError: error 2
    +------------------------------------


In [10]:
try:
    f()
except Exception as e:
    print(f'caught {type(e)}: e')

caught <class 'ExceptionGroup'>: e


### By using `except*` instead of `except`, we can selectively handle only the exceptions in the group that match a certain type. In the following example, which shows a nested exception group, each `except*` clause extracts from the group exceptions of a certain type while letting all other exceptions propagate to other clauses and eventually to be reraised.

In [11]:
def f():
    raise ExceptionGroup(
        "group1",
        [
            OSError(1),
            SystemError(2),
            ExceptionGroup(
                "group2",
                [
                    OSError(3),
                    RecursionError(4)
                ]
            )
        ]
    )

try:
    f()
except* OSError as e:
    print("There were OSErrors")
except* SystemError as e:
    print("There were SystemErrors")

There were OSErrors
There were SystemErrors


  + Exception Group Traceback (most recent call last):
  |   File "/nsls2/conda/envs/2025-3.0-py312-tiled/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_976887/1308333313.py", line 18, in <module>
  |     f()
  |   File "/tmp/ipykernel_976887/1308333313.py", line 2, in f
  |     raise ExceptionGroup(
  | ExceptionGroup: group1 (1 sub-exception)
  +-+---------------- 1 ----------------
    | ExceptionGroup: group2 (1 sub-exception)
    +-+---------------- 1 ----------------
      | RecursionError: 4
      +------------------------------------


Note that the exceptions nested in an exception group must be instances, not types. This is because in practice the exceptions would typically be ones that have already been raised and caught by the program, along the following pattern:

In [12]:
excs = []
for test in tests:
    try:
        test.run()
    except Exception as e:
        excs.append(e)

if excs:
   raise ExceptionGroup("Test Failures", excs)

NameError: name 'tests' is not defined

---
### **Enriching Exceptions with Notes**
When an exception is created in order to be raised, it is usually initialized with information that describes the error that has occurred. There are cases where it is useful to add information after the exception was caught. For this purpose, exceptions have a method `add_note(note)` that accepts a string and adds it to the exception’s notes list. The standard traceback rendering includes all notes, in the order they were added, after the exception.

In [13]:
try:
    raise TypeError('bad type')
except Exception as e:
    e.add_note('Add some information')
    e.add_note('Add some more information')
    raise

TypeError: bad type

### For example, when collecting exceptions into an exception group, we may want to add context information for the individual errors. In the following each exception in the group has a note indicating when this error has occurred.

In [14]:
def f():
    raise OSError('operation failed')

excs = []
for i in range(3):
    try:
        f()
    except Exception as e:
        e.add_note(f'Happened in Iteration {i+1}')
        excs.append(e)

raise ExceptionGroup('We have some problems', excs)

  + Exception Group Traceback (most recent call last):
  |   File "/nsls2/conda/envs/2025-3.0-py312-tiled/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_976887/1429863547.py", line 12, in <module>
  |     raise ExceptionGroup('We have some problems', excs)
  | ExceptionGroup: We have some problems (3 sub-exceptions)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/tmp/ipykernel_976887/1429863547.py", line 7, in <module>
    |     f()
    |   File "/tmp/ipykernel_976887/1429863547.py", line 2, in f
    |     raise OSError('operation failed')
    | OSError: operation failed
    | Happened in Iteration 1
    +---------------- 2 ----------------
    | Traceback (most recent call last):
    |   File "/tmp/ipykernel_976887/1429863547.py", line 7, in <module>
    |     f()
    |   File "/tmp/ipykernel_976887/1429863547.py"